# 0) Load everything

## Imports

In [ ]:
import os
import random

import numpy as np
import pandas as pd
import scipy.io as sio # for loading MATLAB files
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() #sets the matplotlib style to seaborn style

from scipy.io import loadmat 
from scipy.ndimage import convolve1d
from scipy.signal import butter, filtfilt, iirnotch, sosfiltfilt, welch, resample, hilbert
from scipy.stats import mode

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, SelectPercentile, chi2

## Load raw data

In [ ]:
raw_npulse_data_path = os.path.join("data", "npulse", "raw")
os.makedirs(raw_npulse_data_path, exist_ok=True)

for root, d_names, f_names in os.walk(raw_npulse_data_path):
    print('File names:', f_names)

list_npulse_data = {}
for file in f_names:
    print(f"Loading {file}...")
    raw_data = pd.read_csv(os.path.join(raw_npulse_data_path, file))
    list_npulse_data[file] = raw_data

## Load cleaned data (if existing)

In [ ]:
npulse_cleaned_data_path = os.path.join("data", "npulse", "cleaned")
os.makedirs(npulse_cleaned_data_path, exist_ok=True)

for root, d_names, f_names in os.walk(npulse_cleaned_data_path):
    print('File names:', f_names)

list_npulse_cleaned_data = {}
for file in f_names:
    print(f"Loading {file}...")
    cleaned_data = pd.read_csv(os.path.join(npulse_cleaned_data_path, file))
    list_npulse_cleaned_data[file] = cleaned_data

## Divide data into training and testing (for realism)

In [ ]:
# Divide the data into training and testing sets
dataset_size = len(list_npulse_cleaned_data)


test_size = int(dataset_size * 0.2)  # 20% for testing
train_size = dataset_size - test_size  # 80% for training

# Take random Files from the list
train_files_id = random.sample(list_npulse_cleaned_data.keys(), k=train_size)
test_files_id = [key for key in list_npulse_cleaned_data.keys() if key not in train_files_id]

print(f"Train files: {train_files_id}")
print(f"Test files: {test_files_id}")

# Concatenate the dataframes
train_data = pd.concat([list_npulse_cleaned_data[key] for key in train_files_id], ignore_index=True)
test_data = pd.concat([list_npulse_cleaned_data[key] for key in test_files_id], ignore_index=True)

print(f"Train data shape: {train_data.head()}")
print(f"Test data shape: {test_data.head()}")

# 1) Preprocessing

categories :
- EMG envelope : LP filter on rectified OR RMS on raw
- Teager-Kaiser Energy Operator (TKEO) : on raw
- Wavelet transform (WT) : on raw
- Others : Kalman filter, MCO, MOO, MOOGA, ALED,...

## Filtering : band-pass filter (20-450Hz)

## Normalization

In [ ]:
# Normalization
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# 2) Model 1 : Thresholding

3 categories : 
- single threshold (ST)
- double threshold (DT)
- adaptive threshold (AT) : direct on raw emg, using SNR

consideration for threshold : 
- std
- period of time
- % max voluntary contraction (%MVC)
- % max emg amplitude

## CWT Feature

In [ ]:
# Mother wavelet (shape of MUAPs)

In [ ]:
# Apply CWT to the data
def cwt(data, wavelet, scales):
    """
    Perform Continuous Wavelet Transform (CWT) on the data using the specified wavelet and scales.
    
    Parameters:
        data (array-like): Input data to transform.
        wavelet (function): Wavelet function to use for the transform.
        scales (array-like): Scales at which to compute the CWT.
    
    Returns:
        array: CWT coefficients.
    """
    cwt_coefficients = np.zeros((len(scales), len(data)))
    for i, scale in enumerate(scales):
        cwt_coefficients[i, :] = wavelet(data, scale)
    return cwt_coefficients

In [ ]:
# Manifestation variable : maximize across scales
def max_cwt(cwt_coefficients):
    """
    Compute the maximum CWT coefficients across scales.
    
    Parameters:
        cwt_coefficients (array): CWT coefficients.
    
    Returns:
        array: Maximum CWT coefficients across scales.
    """
    return np.max(cwt_coefficients, axis=0)


## Thresholding

In [ ]:
# Threshold coefficient gamma based on SNR (bias & std)



In [ ]:
# Activity detection
def detect_activity(data, threshold):
    """
    Detect activity in the data based on a threshold.
    
    Parameters:
        data (array-like): Input data to analyze.
        threshold (float): Threshold for activity detection.
    
    Returns:
        array: Boolean array indicating detected activity.
    """
    return np.abs(data) > threshold

# 3) Model 2 : Classification